In [2]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating new environment at `~/SSD/Code/TinyMPC/julia/data_gen/Project.toml`
  No Changes to `~/SSD/Code/TinyMPC/julia/data_gen/Project.toml`
  No Changes to `~/SSD/Code/TinyMPC/julia/data_gen/Manifest.toml`


In [4]:
using Symbolics
using LinearAlgebra
import ForwardDiff as FD

In [16]:
function discrete_dynamics(x,u)
    # RK4
    A = [1.0  0.0  0.1  0.0;
         0.0  1.0  0.0  0.1;
         0.0  0.0  1.0  0.0;
         0.0  0.0  0.0  1.0]
    B = [0.005  0.0;
         0.0    0.005;
         0.1    0.0;
         0.0    0.1]
    return A*x + B*u
end
x0 = [5,7,2,-1.4]
N = 3
U = [rand(2) for k=1:N-1]
X = [deepcopy(x0) for k=1:N]
for k=1:N-1
    X[k+1] = discrete_dynamics(X[k],U[k])
end

In [18]:
# Export a vector of vectors to C header and source 
function export_vec_to_c(filename, var_type, var_name, data)
    declare = var_type * " " * var_name
    size = length(data)*length(data[1])
    def = declare * "[" * string(size) * "] = {\n"
    for i=1:length(data)
        def = def * "  "
        for j=1:length(data[1])
            def = def * string(data[i][j])
            if j < length(data[1]) 
                def = def * ","
            end
        end
        if i < length(data)
            def = def * ",\n"
        end
    end
    def = def*"}"

    # open(filename*".h","a") do io
    #         println(io, declare * ";\n")
    # end
    open(filename*".h","a") do io
            println(io, def * ";\n")
    end
    return true
end
export_vec_to_c("data", "sfloat", "x_data", X)
export_vec_to_c("data", "sfloat", "u_data", U)

true